In [4]:
# Install the schedule module if not already installed
#!pip install schedule

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import schedule
import time
import datetime

# Your BLS API Key
API_KEY = 'YOUR_BLS_API_KEY'

# BLS API Endpoint
BLS_API_URL = "https://api.bls.gov/publicAPI/v2/timeseries/data/"

# Series IDs for data
SERIES_IDS = ["CEU0000000001", "LNS14000000"]  # Non-farm workers and unemployment rate

# Request headers
headers = {
    'Content-type': 'application/json'
}

def fetch_bls_data(series_ids):
    payload = {
        "seriesid": series_ids,
        "startyear": "2010",
        "endyear": str(datetime.datetime.now().year),
        "registrationkey": API_KEY
    }
    response = requests.post(BLS_API_URL, json=payload, headers=headers)
    return response.json()

def parse_bls_data(data):
    series_data = {}
    for series in data['Results']['series']:
        series_id = series['seriesID']
        series_data[series_id] = pd.DataFrame(series['data'])
        series_data[series_id]['value'] = pd.to_numeric(series_data[series_id]['value'])
        series_data[series_id]['year'] = pd.to_numeric(series_data[series_id]['year'])
        series_data[series_id]['period'] = series_data[series_id]['period'].apply(lambda x: int(x.lstrip('M')))
        series_data[series_id]['date'] = pd.to_datetime(series_data[series_id]['year'].astype(str) + '-' + series_data[series_id]['period'].astype(str).str.zfill(2))
        series_data[series_id].sort_values('date', inplace=True)
    return series_data

def update_dashboard():
    data = fetch_bls_data(SERIES_IDS)
    parsed_data = parse_bls_data(data)

    # Example plot for non-farm workers
    plt.figure(figsize=(10, 6))
    plt.plot(parsed_data['CEU0000000001']['date'], parsed_data['CEU0000000001']['value'])
    plt.title('Total Non-farm Workers')
    plt.xlabel('Date')
    plt.ylabel('Number of Workers')
    plt.grid(True)
    plt.show()

    # Example plot for unemployment rate
    plt.figure(figsize=(10, 6))
    plt.plot(parsed_data['LNS14000000']['date'], parsed_data['LNS14000000']['value'])
    plt.title('Unemployment Rate')
    plt.xlabel('Date')
    plt.ylabel('Unemployment Rate (%)')
    plt.grid(True)
    plt.show()

# Helper function to schedule task monthly
def job():
    if datetime.datetime.now().day == 1:
        update_dashboard()

# Schedule the data collection for the first of every month
schedule.every().day.at("00:00").do(job)





Every 1 day at 00:00:00 do job() (last run: [never], next run: 2024-12-11 00:00:00)